In [1]:
# you have to install twarc if you havent
# pip install --upgrade twarc (i did it through cmd)

from twarc import Twarc2, expansions
from os import listdir
from os.path import isfile, join
from pprint import pprint

import pandas as pd
import concurrent.futures
import json
import csv
import re

TWEET_IDS_FOLDER_PATH = "./TweetIds/"
OUTPUT_FOLDER_PATH = "./Output/"
BEARER_TOKEN = "AAAAAAAAAAAAAAAAAAAAAIL%2BUQEAAAAA8D63uvPE1VQ1JhYekfOspWUpASE%3DAPe0SIYzJTyfHbUDHB627CHZAKurxidbGo0EqviSS0gi0dUySE"

In [2]:
def loadTweetIds(start_range, end_range, path=TWEET_IDS_FOLDER_PATH):
        print("Loading tweet IDs...")

        full_list = [f for f in listdir(path) if isfile(join(path, f)) and f[-4:] == ".csv"]
        file_names = full_list[start_range:end_range]
        temp_list = []
        count = 0
        for file_name in file_names:
            print(file_name)
            temp_list += pd.read_csv(
                path + file_name,
                index_col=False,
                header=0,
                squeeze = True,
                lineterminator='\n').to_list()

            count += 1
            progress = count / len(file_names) * 100
            print("Progress: %.2f%%" % progress)

        print("%d tweet IDs have been loaded into memory." % len(temp_list))

        return temp_list

def initializeClient():
    return Twarc2(bearer_token=BEARER_TOKEN)

In [3]:
# Load tweetids from csv
# 12, 13 is file 3 (mali kasi yung order sa folder)
# 0, 1, 10, 11, ..., 2, 3, 4, ....
start = 12
end = 13
filename = 'tweets3utf8.csv' # ngayon palang, iset mo na yung file name
tweet_ids = loadTweetIds(start,end)

Loading tweet IDs...
tweet_ids_3.csv
Progress: 100.00%
1000000 tweet IDs have been loaded into memory.


In [4]:
# tweet_ids.sort(reverse=True)
# for i in range(0, 33):
#     tweet_ids.pop(0)
# tweet_ids.sort()
# for i in range(0, 40):
#     tweet_ids.pop(0)
# df = pd.DataFrame(tweet_ids).reset_index(drop=True)
# df.to_csv('tweet_id_1_clean.csv', index=False)

In [5]:
%%time
# Readying the Twitter client to interface
# Note:
client = initializeClient()
lookup = client.tweet_lookup(tweet_ids=tweet_ids[:1000000])

# Here is where all results are placed. Kindly modify this
# so that you can save the tweets to your own machine. In its
# current form, this simply takes all tweets and saves some 
# information to a tweet
tweets = []

for page in lookup:
    # This returns a list of tweets
    result = expansions.flatten(page)
    # For each tweet that was found, add it to a list. For
    # this example, I'm only taking the text and when the tweet
    # was created at.
    for tweet in result:
        tweets.append({
                'text': tweet['text'],
                'date': tweet['created_at'],
                'lang': tweet['lang'],
                'author': tweet['author_id']
            })
    
tweets_df = pd.DataFrame.from_dict(tweets)

rate limit exceeded: sleeping 104.08223176002502 secs
rate limit exceeded: sleeping 110.98654580116272 secs
rate limit exceeded: sleeping 109.67781662940979 secs
rate limit exceeded: sleeping 116.29285883903503 secs
rate limit exceeded: sleeping 107.45628571510315 secs
rate limit exceeded: sleeping 114.1849935054779 secs
rate limit exceeded: sleeping 109.43475389480591 secs
rate limit exceeded: sleeping 116.428626537323 secs
rate limit exceeded: sleeping 109.00942087173462 secs
rate limit exceeded: sleeping 114.10310125350952 secs
rate limit exceeded: sleeping 112.4441225528717 secs


Wall time: 2h 48min 19s


In [6]:
len(tweets_df)

552858

In [7]:
tweets_df.head()

,text,date,lang,author
0,download lang nang download lang nang download...,2020-04-08T01:37:02.000Z,tl,3310177483
1,beberexha asking txt for a collab yaaaaaaaz,2020-04-08T01:37:03.000Z,en,1906684830
2,We all deserve 24 hours jowa challenge after q...,2020-04-08T01:37:03.000Z,en,1016671628229349376
3,new username: 👆🏻,2020-04-08T01:37:03.000Z,en,335193954
4,Yoko na mag puyat .last na yun,2020-04-08T01:37:04.000Z,tl,3062232246


In [8]:
%%time
tweets_df.to_csv(filename, encoding='utf-8', quoting=csv.QUOTE_ALL)
# save raw dataset to csv

Wall time: 2.63 s


In [9]:
# check if you can read properly the new csv

#fn = 'tweets1.csv'
temp_df = pd.read_csv(filename, index_col=0)
temp_df.head()

,text,date,lang,author
0,download lang nang download lang nang download...,2020-04-08T01:37:02.000Z,tl,3310177483
1,beberexha asking txt for a collab yaaaaaaaz,2020-04-08T01:37:03.000Z,en,1906684830
2,We all deserve 24 hours jowa challenge after q...,2020-04-08T01:37:03.000Z,en,1016671628229349376
3,new username: 👆🏻,2020-04-08T01:37:03.000Z,en,335193954
4,Yoko na mag puyat .last na yun,2020-04-08T01:37:04.000Z,tl,3062232246


In [10]:
# def single_thread(pages, start, end):
#     tweets = []
#     for x in range(start, end):
#         result = expansions.flatten(pages[x])
#         for tweet in result:
#             tweets.append({
#                     'text': tweet['text'],
#                     'date': tweet['created_at'],
#                     'lang': tweet['lang'],
#                     'author': tweet['author_id']
#                 })
            
#     return tweets

In [11]:
# %%time

# start = 12
# end = 13
# tweet_ids = loadTweetIds(start,end)
    
# # Readying the Twitter client to interface
# # Note:
# client = initializeClient()
# lookup = client.tweet_lookup(tweet_ids=tweet_ids[:10000])

# pages = []
# for page in lookup:
#     pages.append(page)

# start = 0
# num_total = len(pages)
# workers = 4

# excess = num_total % workers 
# split = int(num_total / workers)
# ranges = []

# for x in range(0, workers):
#     if excess > 0:
#         end = start + split + 1
#         excess -= 1
#     else:
#         end = start + split
    
#     ranges.append([start, end])
#     start = end

# with concurrent.futures.ThreadPoolExecutor(max_workers = workers) as executor:
#     tweet_segments = executor.map(lambda arg_set : single_thread(pages, *arg_set), ranges)

# tweets = []

# for ts in tweet_segments:
#     tweets = tweets + ts
    
# tweets_df = pd.DataFrame.from_dict(tweets)